In [1]:
import math
import os
import re
from collections import Counter, defaultdict
import numpy as np
def read_20_newsgroups(directory):
    print(f' Reading 20 Newsgroups dataset from {directory}...')
    word_counter = Counter()
    dataset = {}
    document_count = defaultdict(int)  
    total_documents = 0 

    tf = defaultdict(list) 

    for curr_dir, classes, files in os.walk(directory):
        curr_class = curr_dir.rsplit('/', 1)[-1]
        dataset[curr_class] = []
        # process each file, take the word and document frequency and store them in the dataset
        for file in files:
            total_documents += 1
            file_path = os.path.join(curr_dir, file)
            read_file = []
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    for line in f:
                        bad = ['Newsgroup', 'document_id', 'From', 'Subject']
                        if any(line.startswith(badd) for badd in bad):
                            continue
                        
                        
                        words = re.findall(r'\b\w+\b', line.lower())  
                        word_counter.update(words)  # Update the counter with words from the line
                        read_file.extend(words)
                        
                    unique_words = set(read_file)  # Get unique words for this document
                    for word in unique_words:
                        document_count[word] += 1  # Increment DF for the word

            except Exception as e:
                print(f"Error reading {file_path}: {e}")
            dataset[curr_class].append(read_file)
    print(f' Finished reading {total_documents} documents from {len(dataset)} classes.')
    # Remove top 300 most common words (stop words)
    stop_words = set([word[0] for word in word_counter.most_common(300)])
    del dataset['20_newsgroups']
    for c, files in dataset.items():
        for file in files:
            file[:] = [word for word in file if word not in stop_words]

    # Use the next 500 most common words
    next_500_words = set(word[0] for word in word_counter.most_common(800)[300:800])
    for c, files in dataset.items():
        for file in files:
            file[:] = [word for word in file if word in next_500_words]
    print(f'finished sorting out the words')
    # Compute and store TF and DF
    for c, files in dataset.items():
        for file in files:
            word_freq_in_file = Counter(file)
            total_words_in_file = len(file)
            
            file_tf = {}  # To store TF for this document
            
            for word in file:
                # Calculate TF for the current word
                tf_value = word_freq_in_file[word] / total_words_in_file
                file_tf[word] = tf_value
            
            # Append the TF for this document to the list for the class
            tf[c].append(file_tf)

    # DF is already tracked in `document_count`, which is the document frequency
    df = document_count

    # Now compute TF-IDF for each word in each document
    tf_idf = defaultdict(list)
    N = total_documents  # Total number of documents

    for c, files in dataset.items():
        for file_tf in tf[c]:
            
            # Initialize a vector of zeros with a length equal to next_500_words
            tfidf_vector = np.zeros(len(next_500_words))
            
            # Create an index mapping for the next_500_words
            word_to_index = {word: idx for idx, word in enumerate(next_500_words)}
            
            # For each word in the document, compute TF-IDF and place it in the correct position
            for word, tf_value in file_tf.items():
                if word in next_500_words:
                    df_value = df[word]  # Get document frequency for the word
                    idf_value = math.log10(N / df_value)  # Calculate IDF
                    tfidf_value = math.log10(1 + tf_value) * idf_value  # Apply the TF-IDF formula
                    
                    # Place the TF-IDF value in the corresponding index
                    tfidf_vector[word_to_index[word]] = tfidf_value
            
            # Append the fixed-length TF-IDF vector to the list for the class
            tf_idf[c].append(tfidf_vector)

    print(f' Finished computing TF-IDF for {N} documents.')
    # print(f'TF-IDF : \n {tf_idf}')
    # print(f'TF : \n {tf}')
    # print(f'DF : \n {df}')
    C = list(dataset.keys())  # List of classes
    return C, tf, df, tf_idf, dataset # treat tf_idf as w

# Call the function
Classes, tf, df, tf_idf, D = read_20_newsgroups("/home/jems/cmsc422/p1/20_newsgroups")


 Reading 20 Newsgroups dataset from /home/jems/cmsc422/p1/20_newsgroups...
 Finished reading 19997 documents from 21 classes.
finished sorting out the words
 Finished computing TF-IDF for 19997 documents.


In [2]:
for c in tf_idf:
    print(f'{c} : {len(tf_idf[c])}')

# soc.religion.christian has 3 less documents than the rest so we append 3 filler arrays to make it 500
filler = np.zeros(500)
for i in range(3):
    tf_idf['soc.religion.christian'].append(filler)

for c in tf_idf:
    for file in tf_idf[c]:
        if len(file) != 500:
            print(f'{c} : {len(file)}')

sci.crypt : 1000
sci.space : 1000
comp.sys.mac.hardware : 1000
soc.religion.christian : 997
talk.religion.misc : 1000
talk.politics.misc : 1000
comp.os.ms-windows.misc : 1000
comp.windows.x : 1000
misc.forsale : 1000
comp.sys.ibm.pc.hardware : 1000
rec.sport.hockey : 1000
rec.motorcycles : 1000
comp.graphics : 1000
rec.sport.baseball : 1000
sci.electronics : 1000
sci.med : 1000
talk.politics.guns : 1000
talk.politics.mideast : 1000
alt.atheism : 1000
rec.autos : 1000


In [3]:
print(f'{len(Classes)}')

print(f'{len([np.array(tf_idf[c]) for c in Classes])}')

20
20


In [4]:
class_dict = {c: i for i, c in enumerate(Classes)} # dictionary to convert class names to indices

temp = np.array([np.array(tf_idf[c]) for c in Classes]) # convert the dictionary to a numpy array

print({len(temp[i]) for i in range(len(temp))})
# At this stage our X has the shape (20, 1000, 500) where 20 is the number of classes, 1000 is the number of documents and 500 is the number of words
# We need to convert this to a shape of (20, 1000) to use in our caculations and we do this by taking the dot product of the word vectors for each document
X = []
for i in range(len(temp)):
    dot_product = []
    for j in range(len(temp[i])):
        dot_product.append(np.dot(temp[i][j], temp[i][j]))
    X.append(np.array(dot_product))
X = np.array(X)
print(X.shape)
# After converting our X into a 2D matrix, we split the data into training and testing data
train, test = [], []
for i in range(len(X)):
    train.append(X[i][len(X[i])//2:])
    test.append(X[i][:len(X[i])//2])

train = np.array(train)
test = np.array(test)

# We set up our classification into a one hot encoding format allowing us to easily access the class of a document
Y = np.full((len(Classes), len(Classes)), -1)
for i in range(len(Classes)):
    Y[i][i] = 1


{1000}
(20, 1000)


In [48]:
from qpsolvers import solve_qp
import osqp
from scipy import sparse

# Here we define our SVM training function
def train_svm(X, Y, C):
    samples, features = X.shape
    Y = Y.astype(float)
    P = np.outer(Y, Y) * np.dot(X, X.T) 
    epsilon = 1e-5
    P += np.eye(samples) * epsilon
    P_sparse = sparse.csr_matrix(P)
    G = np.vstack((np.eye(samples) * -1, np.eye(samples)))
    G_sparse = sparse.csr_matrix(G)
    H = np.vstack((np.zeros(samples), np.ones(samples) * C)).flatten()
    Q = np.full(samples, -1)
    A = Y.reshape(1, -1)
    A_sparse = sparse.csr_matrix(A)
    b = np.array([0.0])
    
    # we extract our supporting vectors and their corresponding alphas and labels
    alphas = solve_qp(P_sparse, Q, G_sparse, H, A_sparse, b, solver='osqp')
    vector_indexes = alphas > 1e-5
    support_alphas = alphas[vector_indexes]
    support_vectors = X[vector_indexes]
    support_labels = Y[vector_indexes]
    
    w = np.sum((support_alphas * support_labels).reshape(-1, 1) * support_vectors, axis=0)
    b = np.mean(support_labels - support_vectors @ w)

    return w, b, support_vectors, support_alphas, support_labels




In [53]:
# We define our slack variable and c and then for every class we train an SVM
c = float(100)
train_svm_per_class = {}
for i, clas in enumerate(Classes):
    w, b, s_vectors, s_alphas, s_labels = train_svm(train, Y[i], c)
    print(f'{Y[i]}')
    train_svm_per_class[clas] = {'w': w, 'b': b, 's_vectors': s_vectors, 's_alphas': s_alphas, 's_labels': s_labels}


[ 1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1]
[-1 -1 -

In [50]:
for clas, svm in train_svm_per_class.items():
    print(f'{clas} : {svm["s_alphas"]}')

sci.crypt : [47.14783055  1.80215108  0.41143787 14.66593773  1.7354826   1.56876368
  0.19266712  1.45775464  2.53560914  0.70473273  4.27977174  1.98165104
  0.70621586  2.44480741  1.86878985  5.71154535  1.95500757  2.16404944
  0.96144874]
sci.space : [1.79611079e+00 2.79641236e+01 9.41615617e-03 2.67023062e+00
 2.53933994e+00 1.82636895e+00 2.06221874e+00 1.06966383e+00
 2.27596478e+00 6.07117751e-01 1.97133755e+00 4.15075295e-01
 1.55683896e+00 2.68946843e+00 1.69987262e+00 2.92950677e+00
 7.57513980e-01 5.82301115e-01 5.05768953e-01]
comp.sys.mac.hardware : [ 0.25722231 27.27529844  0.29942356  1.665049    0.81454482  0.84192445
  3.51932311  1.6980517   3.57773812  1.35153916  2.33481985  2.50084184
  0.0313701   3.03406452  0.20892071  1.27923057  2.74569405  1.11556396]
soc.religion.christian : [1.37449556e+01 1.75364058e+00 5.48962222e+01 4.97283210e+00
 1.26106646e+01 1.27918358e-01 4.80895785e-01 3.36202509e+00
 4.06665986e-02 9.41436027e+00 3.28800691e+00 4.06392727e+00


In [51]:
# for every class we predict the SVM
def predict_svm(X, w, b):
    return X @ w + b

predict_svm_per_class = {}
for clas, svm in train_svm_per_class.items():
    predict_svm_per_class[clas] = lambda X, svm=svm: predict_svm(test, svm['w'], svm['b'])

In [52]:
# We check the acccuracy of our SVM
correct = 0
total = 0
for clas, svm in predict_svm_per_class.items():
    total += 1
    result = np.argmax(svm(test))
    if class_dict[clas] == result:
        correct += 1
    print(f'{clas} : {result}')
    print(f'Correct score = {class_dict[clas]}')
print(f'Accuracy: {correct/total}') 

sci.crypt : 4
Correct score = 0
sci.space : 8
Correct score = 1
comp.sys.mac.hardware : 12
Correct score = 2
soc.religion.christian : 3
Correct score = 3
talk.religion.misc : 18
Correct score = 4
talk.politics.misc : 15
Correct score = 5
comp.os.ms-windows.misc : 0
Correct score = 6
comp.windows.x : 10
Correct score = 7
misc.forsale : 6
Correct score = 8
comp.sys.ibm.pc.hardware : 6
Correct score = 9
rec.sport.hockey : 0
Correct score = 10
rec.motorcycles : 15
Correct score = 11
comp.graphics : 8
Correct score = 12
rec.sport.baseball : 8
Correct score = 13
sci.electronics : 4
Correct score = 14
sci.med : 1
Correct score = 15
talk.politics.guns : 6
Correct score = 16
talk.politics.mideast : 10
Correct score = 17
alt.atheism : 6
Correct score = 18
rec.autos : 4
Correct score = 19
Accuracy: 0.05


In [56]:
# we repeat these steps for the kernelized SVM
def svm_with_kernel(X, Y, C, c=0, d=2):
    poly_kernel = lambda x, y, c, d: (np.dot(x, y.T) + c) ** d
    samples, features = X.shape
    Y = Y.astype(float)
    P = np.outer(Y, Y) * poly_kernel(X, X, c, d)
    epsilon = 1e-5
    P += np.eye(samples) * epsilon
    P_sparse = sparse.csr_matrix(P)
    G = np.vstack((np.eye(samples) * -1, np.eye(samples)))
    G_sparse = sparse.csr_matrix(G)
    H = np.vstack((np.zeros(samples), np.ones(samples) * C)).flatten()
    Q = np.full(samples, -1)
    A = Y.reshape(1, -1)
    A_sparse = sparse.csr_matrix(A)
    b = np.array([0.0])
    
    alphas = solve_qp(P_sparse, Q, G_sparse, H, A_sparse, b, solver='osqp')
    vector_indexes = alphas > 1e-5
    support_alphas = alphas[vector_indexes]
    support_vectors = X[vector_indexes]
    support_labels = Y[vector_indexes]
    
    w = np.sum((support_alphas * support_labels).reshape(-1, 1) * support_vectors, axis=0)
    b = np.mean(support_labels - support_vectors @ w)

    return w, b, support_vectors, support_alphas, support_labels

In [57]:
c = float(100)
kernel_svm_per_class = {}
for i, clas in enumerate(Classes):
    w, b, s_vectors, s_alphas, s_labels = svm_with_kernel(train, Y[i], c)
    kernel_svm_per_class[clas] = {'w': w, 'b': b, 's_vectors': s_vectors, 's_alphas': s_alphas, 's_labels': s_labels}

predict_kernel_svm_per_class = {}
for clas, svm in kernel_svm_per_class.items():
    predict_kernel_svm_per_class[clas] = lambda X, svm=svm: predict_svm(test, svm['w'], svm['b'])

correct = 0
total = 0
for clas, svm in predict_kernel_svm_per_class.items():
    total += 1
    result = np.argmax(svm(test))
    if class_dict[clas] == result:
        correct += 1
    print(f'{clas} : {result}')
    print(f'Correct score = {class_dict[clas]}')
print(f'Accuracy: {correct/total}') 

sci.crypt : 13
Correct score = 0
sci.space : 8
Correct score = 1
comp.sys.mac.hardware : 12
Correct score = 2
soc.religion.christian : 3
Correct score = 3
talk.religion.misc : 6
Correct score = 4
talk.politics.misc : 15
Correct score = 5
comp.os.ms-windows.misc : 0
Correct score = 6
comp.windows.x : 10
Correct score = 7
misc.forsale : 6
Correct score = 8
comp.sys.ibm.pc.hardware : 6
Correct score = 9
rec.sport.hockey : 0
Correct score = 10
rec.motorcycles : 15
Correct score = 11
comp.graphics : 8
Correct score = 12
rec.sport.baseball : 14
Correct score = 13
sci.electronics : 4
Correct score = 14
sci.med : 1
Correct score = 15
talk.politics.guns : 6
Correct score = 16
talk.politics.mideast : 10
Correct score = 17
alt.atheism : 6
Correct score = 18
rec.autos : 4
Correct score = 19
Accuracy: 0.05
